# 3D Transformations

## Packages:

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import numpy as np
import ipywidgets as widgets
import json

## Utilities:

In [ ]:
def mesh2D(xlim, ylim, n=5): #Set up Meshgrid
    if isinstance(n, int):
        x = np.linspace(xlim[0],xlim[1],n)
        y = np.linspace(ylim[0],ylim[1],n)
    elif isinstance(n, list):
        x = np.linspace(xlim[0],xlim[1],n[0])
        y = np.linspace(ylim[0],ylim[1],n[1])
    else:
        raise Exception("Invalid Parameter")
        
    return np.meshgrid(x, y, sparse=True)

In [ ]:
def normalize(v): #for Evec, Eval
    magnitude = np.sqrt(v[0]**2 + v[1]**2 + v[2]**2)
    if magnitude==0:
        raise ValueError("Zero vector cannot be normalized.")
    else:
        return v/magnitude

## Objects:

In [ ]:
class Line: 
    def __init__(self, pointList):
        self.x = []
        self.y = []
        self.z = []
        
        for i in range(len(pointList)):
            self.x.append(pointList[i][0])
            self.y.append(pointList[i][1])
            self.z.append(pointList[i][2])
        
    def gObject(self, name='', color='rgb(210,64,0)'):
        lineObject = go.Scatter3d(name=name,
                                  mode="lines",
                                  x=self.x,
                                  y=self.y,
                                  z=self.z,
                                  line=dict(color=(color),
                                            width=7)
                                 )
        return lineObject

In [ ]:
class Sphere:
    def __init__(self, radius=5, center=[0, 0, 0]):
        self.radius = radius
        self.center = center
        meshSize = 20
        theta = np.linspace(0,2*np.pi,meshSize)
        phi = np.linspace(0,np.pi,meshSize)
        self.x = radius*np.outer(np.cos(theta),np.sin(phi)) + center[0]
        self.y = radius*np.outer(np.sin(theta),np.sin(phi)) + center[1]
        self.z = radius*np.outer(np.ones(meshSize),np.cos(phi)) + center[2]
        
    def gObject(self, name='Sphere', color=[[0.0, 'rgb(0,62,116)'], [1.0, 'rgb(255,255,255)']]):
        sphere = go.Surface(name=name,
                            x=self.x,
                            y=self.y,
                            z=self.z,
                            showscale=False,
                            opacity=0.7,
                            colorscale=color
                           )       
        return sphere

In [ ]:
class Point:
    def __init__(self, position):
        self.position = np.array(position)
    
    def gObject(self, name='', color='rgb(0,62,116)'):
        point = go.Scatter3d(mode="markers",
                             name=name,
                             x=[self.position[0]],
                             y=[self.position[1]],
                             z=[self.position[2]],
                             marker=dict(color=color,
                                         size=7
                                        )
                            )
        return point

Orange: 'rgb(210,64,0)'
Dark Green: 'rgb(2,137,59)'
Imperial Blue: 'rgb(0,62,116)'
Pool Blue: 'rgb(2,161,205)'

## 3D Visualization

### Unit Vectors and a Sphere

In [ ]:
data = []
for i in range(3):
    uvec = [0, 0, 0]
    uvec[i] = 1
    line = Line([[0, 0, 0], uvec])
    data.append(line.gObject('rgb(0,0,0)'))
    
radius = 2*np.sqrt(3)
sphere = Sphere(radius)
data.append(sphere.gObject())

### Frames

In [ ]:
theta1 = np.pi/2
t = np.linspace(0, theta1, 10)
initialPosition = np.matrix([[2], [2], [2]])

In [ ]:
def roXaxis(theta):
    M = np.matrix([[1, 0, 0],
                   [0, np.cos(theta), -np.sin(theta)], 
                   [0, np.sin(theta), np.cos(theta)]
                  ])
    return M

def roYaxis(theta):
    M = np.matrix([[np.cos(theta), 0, np.sin(theta)],
                   [0, 1, 0],
                   [-np.sin(theta), 0, np.cos(theta)]
                  ])
    return M

def roZaxis(theta):
    M = np.matrix([[np.cos(theta), -np.sin(theta), 0],
                   [np.sin(theta), np.cos(theta), 0],
                   [0, 0 ,1]
                  ])
    return M

In [ ]:
def computeFrames(rotation, theta, point, frames):
    vecPoint = np.matrix([[point[0]],
                          [point[1]],
                          [point[2]]
                         ])
    t = np.linspace(0, theta, frames)
    
    lineList = [point]
    output = []
    for i in t:
        newPoint = rotation(i)*vecPoint
        ptList = np.reshape(newPoint,(1,3)).tolist()[0]
        lineList.append(ptList)
        output.append([Point(ptList).gObject(), Line(lineList).gObject('rgb(210,64,0)')])
    return [output,ptList]

In [ ]:
def computeCompositeFrames(rotation1,rotation2, theta, point, frames):
    vecPoint = np.matrix([[point[0]],
                          [point[1]],
                          [point[2]]
                         ])
    t = np.linspace(0, theta, frames)
    
    lineList = [point]
    output = []
    for i in t:
        newPoint = rotation1(i)*vecPoint
        ptList = np.reshape(newPoint,(1,3)).tolist()[0]
        lineList.append(ptList)
        output.append([Point(ptList).gObject(), Line(lineList).gObject('rgb(210,64,0)')])
    for j in t:
        newPoint2 = rotation2(j)*newPoint
        ptList = np.reshape(newPoint2,(1,3)).tolist()[0]
        lineList.append(ptList)
        output.append([Point(ptList).gObject(), Line(lineList).gObject('rgb(210,64,0)')])
        
    return output

In [ ]:
# frameSize = 10
# frameList = computeFrames(roZaxis, np.pi/2, [2, 2, 2], frameSize)

# data.append(frameList[9][0])
# data.append(frameList[9][1])

# newPoint = [frameList[9][0].x[0],
#             frameList[9][0].y[0],
#             frameList[9][0].z[0]
#            ]

# frameList = computeFrames(roXaxis, np.pi/2, newPoint, frameSize)

# data.append(frameList[9][0])
# data.append(frameList[9][1])

# newPoint = [frameList[9][0].x[0],
#             frameList[9][0].y[0],
#             frameList[9][0].z[0]
#            ]

# frameList = computeFrames(roYaxis, -np.pi/2, newPoint, frameSize)

# data.append(frameList[9][0])
# data.append(frameList[9][1])

frameSize = 10
frameList = computeFrames(roZaxis, np.pi/2, [2, 2, 2], frameSize)

data.append(frameList[5][0])
data.append(frameList[5][1])

In [ ]:
# fig=go.Figure(data=data)
# py.plot(fig)

In [ ]:
# def rotationPlot(frame):
#     global data
#     data = []
    
#     for i in range(3):
#         uvec = [0, 0, 0]
#         uvec[i] = 1
#         line = Line([[0, 0, 0], uvec])
#         data.append(line.gObject('rgb(0,0,0)'))
        
#     radius = 2*np.sqrt(3)
#     sphere = Sphere(radius)
#     data.append(sphere.gObject())
    
#     initialPoint = [2, 2, 2]
    
#     data.append(Point(initialPoint).gObject('rgb(0,0,0)'))
    
#     frameSize = 10
#     frameList1 = computeCompositeFrames(roYaxis,roZaxis,
#                               np.pi/2,
#                               initialPoint, 
#                               frameSize)
#     frameList2 = computeCompositeFrames(roZaxis,roYaxis,
#                               np.pi/2,
#                               initialPoint, 
#                               frameSize)
    
#     data.append(frameList1[frame][0]) 
#     data.append(frameList1[frame][1])
#     data.append(frameList2[frame][0]) 
#     data.append(frameList2[frame][1])

#     py.iplot(data)
    
# widgets.interact(rotationPlot,frame=widgets.IntSlider(min=0,max=19,step=1,value=0,continuous_update=True) )

In [ ]:
data = []
basisName = ["iVec", "jVec", "kVec"]
for i in range(3):
    uvec = [0, 0, 0]
    uvec[i] = 1
    line = Line([[0, 0, 0], uvec])
    data.append(line.gObject(basisName[i], 'rgb(0,0,0)'))
        
radius = 2*np.sqrt(3)
sphere = Sphere(radius)
data.append(sphere.gObject(name='Sphere'))
    
initialPoint = [2, 2, 2]
    
data.append(Point(initialPoint).gObject('initial point', 'rgb(0,0,0)'))

frame = []
    
frameSize = 10
frameList1 = computeCompositeFrames(roYaxis,roZaxis,
                                    np.pi/2,
                                    initialPoint, 
                                    frameSize)
frameList2 = computeCompositeFrames(roZaxis,roYaxis,
                                    np.pi/2,
                                    initialPoint, 
                                    frameSize)
for i in range(frameSize):
    frame.append(dict(data=frameList1[i][0], name="frame %i" %i))    

mainData = dict(data=data, frame=frame)

print(mainData)